# TensorFlow!

In [8]:
import tensorflow
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

client1_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')
client2_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')

# It's training time!

In [9]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = numpy.load('dataset.npz') # Download dataset.npz from the link in Readme.md
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
	return b, v

x_train, y_train = get_dataset()

In [10]:
# Split dataset for two clients
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split

client1_x, client2_x, client1_y, client2y =train_test_split(x_train, y_train, test_size=0.5, random_state=100)

In [11]:
"""
Training client 1 model
"""
from tensorflow.keras.callbacks import ModelCheckpoint

# checkpoint_filepath = '/checkpoint/f0-1/'
# model_checkpointing_callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_best_only=True,
# )
client1_model.fit(client1_x, client1_y,
          batch_size=2048,
          epochs=50,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])


Epoch 1/50
330/330 [==============================] - 152s 461ms/step - loss: 2.8262e-04 - val_loss: 2.9083e-04 - lr: 5.0000e-06
Epoch 2/50
330/330 [==============================] - 154s 467ms/step - loss: 2.8217e-04 - val_loss: 2.9101e-04 - lr: 5.0000e-06
Epoch 3/50
330/330 [==============================] - 154s 468ms/step - loss: 2.8176e-04 - val_loss: 2.9111e-04 - lr: 5.0000e-06
Epoch 4/50
330/330 [==============================] - 155s 470ms/step - loss: 2.8142e-04 - val_loss: 2.9137e-04 - lr: 5.0000e-06
Epoch 5/50
330/330 [==============================] - 151s 459ms/step - loss: 2.8106e-04 - val_loss: 2.9170e-04 - lr: 5.0000e-06
Epoch 6/50
330/330 [==============================] - 151s 457ms/step - loss: 2.8077e-04 - val_loss: 2.9163e-04 - lr: 5.0000e-06
Epoch 7/50
330/330 [==============================] - 150s 455ms/step - loss: 2.8047e-04 - val_loss: 2.9206e-04 - lr: 5.0000e-06
Epoch 8/50
330/330 [==============================] - 152s 461ms/step - loss: 2.8021e-04 - val_lo

In [12]:
"""
Training client 2 model
"""
# checkpoint_filepath = './checkpoint/f0-2'
# model_checkpointing_callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_best_only=True,
# )
client2_model.fit(client2_x, client2y,
          batch_size=2048,
          epochs=50,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])


Epoch 1/50
330/330 [==============================] - 154s 467ms/step - loss: 2.8485e-04 - val_loss: 2.8847e-04 - lr: 5.0000e-06
Epoch 2/50
330/330 [==============================] - 160s 484ms/step - loss: 2.8437e-04 - val_loss: 2.8864e-04 - lr: 5.0000e-06
Epoch 3/50
330/330 [==============================] - 155s 468ms/step - loss: 2.8398e-04 - val_loss: 2.8908e-04 - lr: 5.0000e-06
Epoch 4/50
330/330 [==============================] - 156s 472ms/step - loss: 2.8366e-04 - val_loss: 2.8886e-04 - lr: 5.0000e-06
Epoch 5/50
330/330 [==============================] - 156s 474ms/step - loss: 2.8334e-04 - val_loss: 2.8898e-04 - lr: 5.0000e-06
Epoch 6/50
330/330 [==============================] - 156s 473ms/step - loss: 2.8305e-04 - val_loss: 2.8986e-04 - lr: 5.0000e-06
Epoch 7/50
330/330 [==============================] - 155s 471ms/step - loss: 2.8279e-04 - val_loss: 2.8935e-04 - lr: 5.0000e-06
Epoch 8/50
330/330 [==============================] - 162s 490ms/step - loss: 2.8257e-04 - val_lo

In [35]:
"""
Aggregrate models
"""

client1_weights = client1_model.get_weights()
client2_weights = client2_model.get_weights()

client1_weights = numpy.array(client1_weights, dtype=object)
client2_weights = numpy.array(client2_weights, dtype=object)

client1_weights = 1/2 * client1_weights
client2_weights = 1/2 * client2_weights

aggregated_weights = numpy.add(client1_weights,client2_weights)

client1_model.set_weights(aggregated_weights)
client1_model.save_weights("model_weight/federated0_weights", save_format="h5")